In [ ]:
#!pip3 install opencv-python
#!pip3 install scikit-image

import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

from sklearn.model_selection import train_test_split

from scipy.ndimage import convolve, binary_fill_holes
from scipy.stats import mode

from skimage import io, exposure
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.transform import rotate, hough_line, hough_line_peaks, resize
from skimage.filters import threshold_otsu, sobel, gaussian, threshold_local

import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam


from glob import glob


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# Load images from csv file

path = pd.read_csv('Dataset/cards.csv')
path_df = pd.DataFrame(path)
# For column names that contain space, replace the space with an underscore
path_df.columns = [c.replace(' ', '_') for c in path_df.columns]
path_df.head()

In [ ]:
path_df['suit'] = path_df['labels'].str.split().str[-1]
print(path_df['suit'].unique())

In [ ]:
# Remove rows with jokers
path_df = path_df[~path_df['suit'].str.contains('joker', case=False)]
card_suits = path_df['suit'].unique()
print(card_suits)

In [ ]:
path_df.head()

In [ ]:
path_df = path_df.drop(columns = ['data_set'])
path_df.head()

In [ ]:
path_df['filepaths'] = path_df['filepaths'].apply(lambda x: 'DatasetPreProcessed/' + x)
path_df.head()

In [ ]:
suit_class_mapping = {
    'spades': 0,
    'hearts': 1,
    'diamonds': 2,
    'clubs': 3
}

# Add a new column 'class_index' based on the suit labels
path_df['suits_class_index'] = path_df['suit'].map(suit_class_mapping)
path_df.head()

In [ ]:
# Split dataframe into train and test sets
train_df, test_df = train_test_split(path_df, test_size=0.2, random_state=42)

# Split train set into train and validation sets
test_df, val_df = train_test_split(train_df, test_size=0.5, random_state=42)

## ResNet

In [ ]:
image_size = [224, 224]

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size + [3]))

print(base_model.summary())

In [ ]:
# Don't want to train model because it is already trained.
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
flatten_layer = Flatten()(base_model.output)
prediction  = Dense(4, activation='softmax')(flatten_layer)
model = Model(inputs=base_model.input, outputs=prediction)
print(model.summary())

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
train_data_generator = ImageDataGenerator(rescale = 1. /255,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True)

test_data_generator = ImageDataGenerator(rescale=1. /255)


training_set = train_data_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col='filepaths',
    y_col='suit',
    target_size=(224,224), 
    batch_size=8, 
    class_mode='categorical')

val_set = test_data_generator.flow_from_dataframe(
    dataframe = val_df,
    x_col='filepaths',
    y_col='suit',
    target_size=(224,224), 
    batch_size=8, 
    class_mode='categorical')

test_set = test_data_generator.flow_from_dataframe(
    dataframe = test_df,
    x_col='filepaths',
    y_col='suit',
    target_size=(224,224), 
    batch_size=8, 
    class_mode='categorical')



In [ ]:
result = model.fit(training_set, 
                   validation_data=val_set, 
                   epochs=10,
                   steps_per_epoch=len(training_set),
                   validation_steps=len(val_set))

In [ ]:
#Plot accuracy
plt.plot(result.history['accuracy'], label='train_acc')
plt.plot(result.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
#Plot Loss
plt.plot(result.history['loss'], label='train_loss')
plt.plot(result.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test set
evaluation = model.evaluate(test_set, steps=len(test_set))

# Print the evaluation metrics
print("Loss:", evaluation[0])
print("Accuracy:", evaluation[1])


In [ ]:
# Perform predictions on the test set
predictions = model.predict(test_set, steps=len(test_set))

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)

#Print the predicted labels
# print(predicted_labels[1])
# print(len(predicted_labels))

In [ ]:
# Ground truth labels for the test set
true_labels = test_set.labels
print(true_labels)

# Calculate precision
precision = precision_score(true_labels, predicted_labels, average='weighted')

# Calculate recall
recall = recall_score(true_labels, predicted_labels, average='weighted')

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print the metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)